In [ ]:
import tensorflow as tf
import time, shutil
import graph
import coarsening

%matplotlib inline

#import models
%run -n models.ipynb

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Directories.
flags.DEFINE_string('dir_data', 'data_mnist', 'Directory to store data.')

# Feature graph

In [ ]:
def grid_graph(m, corners=False):
    z = graph.grid(m)
    A = graph.adjacency(z, k=FLAGS.number_edges, metric=FLAGS.metric)

    # Connections are only vertical or horizontal on the grid.
    # Corner vertices are connected to 2 neightbors only.
    if corners:
        import scipy.sparse
        A = A.toarray()
        A[A < A.max()/1.5] = 0
        A = scipy.sparse.csr_matrix(A)
        print('{} edges'.format(A.nnz))

    print("{} > {} edges".format(A.nnz//2, FLAGS.number_edges*m**2//2))
    return A

def coarsen(A, levels):
    graphs, parents = coarsening.metis(A, levels)
    perms = coarsening.compute_perm(parents)

    laplacians = []
    for i,A in enumerate(graphs):
        M, M = A.shape

        # No self-connections.
        if True:
            A = A.tocoo()
            A.setdiag(0)

        if i < levels:
            A = coarsening.perm_adjacency(A, perms[i])

        A = A.tocsr()
        A.eliminate_zeros()
        Mnew, Mnew = A.shape
        print('Layer {0}: M_{0} = |V| = {1} nodes ({2} added), |E| = {3} edges'.format(i, Mnew, Mnew-M, A.nnz//2))

        L = graph.laplacian(A, normalized=FLAGS.normalized_laplacian)
        laplacians.append(L)
    return laplacians, perms[0] if len(perms) > 0 else None

t_start = time.process_time()
A = grid_graph(28, corners=False)
A = graph.replace_random_edges(A, 0)
L, perm = coarsen(A, FLAGS.coarsening_levels)
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
graph.plot_spectrum(L)
del A

# Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(FLAGS.dir_data, one_hot=False)

t_start = time.process_time()
mnist.train._images = coarsening.perm_data(mnist.train._images, perm)
mnist.validation._images = coarsening.perm_data(mnist.validation._images, perm)
mnist.test._images = coarsening.perm_data(mnist.test._images, perm)
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
del perm

# Neural networks

In [ ]:
#model = fc1()
#model = fc2(nhiddens=100)
#model = cnn2(K=5, F=10)  # K=28 is equivalent to filtering with fgcnn.
#model = fcnn2(F=10)
#model = fgcnn2(L[0], F=10)
#model = lgcnn2_2(L[0], F=10, K=10)
#model = cgcnn2_3(L[0], F=10, K=5)
#model = cgcnn2_4(L[0], F=10, K=5)
#model = cgcnn2_5(L[0], F=10, K=5)

if False:
    K = 5  # 5 or 5^2
    t_start = time.process_time()
    mnist.test._images = graph.lanczos(L, mnist.test._images.T, K).T
    mnist.train._images = graph.lanczos(L, mnist.train._images.T, K).T
    model = lgcnn2_1(L, F=10, K=K)
    print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
    ph_data = tf.placeholder(tf.float32, (FLAGS.batch_size, mnist.train.images.shape[1], K), 'data')

In [ ]:
common = {}
common['num_epochs']     = 10
common['batch_size']     = 100
common['decay_steps']    = mnist.train.num_examples / common['batch_size']
common['eval_frequency'] = 30 * common['num_epochs']
common['filter']         = 'chebyshev5'
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
C = max(mnist.train.labels) + 1  # number of classes

shutil.rmtree('summaries/mnist1', ignore_errors=True)
shutil.rmtree('checkpoints/mnist1', ignore_errors=True)

In [ ]:
params = common.copy()
params['dir_name']       = 'mnist1/softmax'
params['regularization'] = 5e-4
params['dropout']        = 1
params['learning_rate']  = 0.03
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['F']              = []
params['K']              = []
params['p']              = []
params['M']              = [C]
model = cgcnn(L, **params)
model.fit(mnist.train.images, mnist.train.labels, mnist.validation.images, mnist.validation.labels)
print('train {}'.format(model.evaluate(mnist.train.images, mnist.train.labels)[0]))
print('test {}'.format(model.evaluate(mnist.test.images, mnist.test.labels)[0]))

In [ ]:
# Corresponds to cgcnn2_2(L[0], F=10, K=20).
params = common.copy()
params['dir_name']       = 'mnist1/gconv_softmax'
params['regularization'] = 0
params['dropout']        = 1
params['learning_rate']  = 0.03
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['F']              = [10]
params['K']              = [20]
params['p']              = [1]
params['M']              = [C]
params['filter']         = 'chebyshev2'
params['brelu']          = 'b2relu'
model = cgcnn(L, **params)
model.fit(mnist.train.images, mnist.train.labels, mnist.validation.images, mnist.validation.labels)
print('train {}'.format(model.evaluate(mnist.train.images, mnist.train.labels)[0]))
print('test {}'.format(model.evaluate(mnist.test.images, mnist.test.labels)[0]))

In [ ]:
# Architecture of TF MNIST conv model (LeNet-5-like).
# Changes: regularization, dropout, decaying training rate, momentum optimizer, stopping condition, size of biases.
# Differences: training data randomization, init conv1 biases at 0.
params = common.copy()
params['dir_name']       = 'mnist1/gconv_gconv_fc_softmax_lenet5'
params['regularization'] = 5e-4
params['dropout']        = 0.5
params['learning_rate']  = 0.03
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['F']              = [32, 64]
params['K']              = [25, 25]
params['p']              = [4, 4]
params['M']              = [512, C]
model = cgcnn(L, **params)
model.fit(mnist.train.images, mnist.train.labels, mnist.validation.images, mnist.validation.labels)
print('train {}'.format(model.evaluate(mnist.train.images, mnist.train.labels)[0]))
print('test {}'.format(model.evaluate(mnist.test.images, mnist.test.labels)[0]))

In [ ]:
# Best performance with architecture 32, 64, 512.
params = common.copy()
params['dir_name']       = 'mnist1/gconv_gconv_fc_softmax_best'
params['regularization'] = 5e-5
params['dropout']        = 0.5
params['learning_rate']  = 0.03
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['F']              = [32, 64]
params['K']              = [20, 20]
params['p']              = [4, 4]
params['M']              = [512, C]
model = cgcnn(L, **params)
model.fit(mnist.train.images, mnist.train.labels, mnist.validation.images, mnist.validation.labels)
print('train {}'.format(model.evaluate(mnist.train.images, mnist.train.labels)[0]))
print('test {}'.format(model.evaluate(mnist.test.images, mnist.test.labels)[0]))